# <font color = 'red'> Pré Processamento e Engenharia de Atributos</font>

### <font color = 'red'> Foco: </font>

- Pré processar o texto para a etapa de análise de sentimentos

- Remoção de stopwords e outros, para verificar ocorrência

In [1]:
# Imports
import os
import re
import numpy as np
import pandas as pd

In [2]:
# Visualizando Cj dados
dados = pd.read_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/data_scraping.csv',
                       sep =',')

In [3]:
dados.head()

,Empresa,Relato
0,Cartão Cencosud GBarbosa,Em (01/11) fui verificar a fatura do cartão de...
1,Latam Airlines (Tam),Minha reserva foi alterada pela empresa e eu t...
2,TCL SEMP,"Prezado, No dia 16/09/2021 fiz a compra de uma..."
3,iFood,Comprei um cobinado de sushi com 60 peças do r...
4,Latam Airlines (Tam),"Comprei as passagens no dia 09/10/24, cancelei..."


In [4]:
instituicoes_financeiras = ['Nubank',
              'Banco do Brasil',
              'Caixa Econômica Federal',
              'Banco Santander',
              'Banco Itaú Unibanco',
              'Mercado Pago',
              'Banco Inter (Banco Intermedium)',
              'Banco Bradesco',
              'C6 Bank']

In [5]:
# Selecionando apenas instituições financeiras

dados_financeiros = dados[dados['Empresa'].isin(instituicoes_financeiras)]

In [6]:
#Verificando ocorrências

dados_financeiros.Empresa.value_counts().reset_index(name='Quantidade Relatos')

,Empresa,Quantidade Relatos
0,Nubank,369
1,Banco do Brasil,268
2,Caixa Econômica Federal,189
3,Banco Santander,135
4,Banco Itaú Unibanco,122
5,Banco Bradesco,108
6,Mercado Pago,104
7,Banco Inter (Banco Intermedium),91
8,C6 Bank,54


In [7]:
dados_financeiros.head(10)

,Empresa,Relato
21,Nubank,Recebi uma proposta do feirao Serasa no valor ...
23,Nubank,"QUERO UM DETALHAMENTO DOS JUROS, QUE ESTÃO MUI..."
28,Nubank,Venho expressar minha insatisfação em relação ...
30,Nubank,"Juros muito altos, pagando muito mais do que o..."
34,Nubank,Entrei em contato com a nubank pelo aplicativo...
35,Nubank,"Prezada equipe do Nubank, Espero que esta mens..."
36,Nubank,"Deixei de pagar uma conta, porem o juros foi a..."
38,Caixa Econômica Federal,"Prezado(a), Venho solicitar, com urgência, a r..."
45,Nubank,Quero renegociar mas com valores dentro da min...
47,Nubank,Ofereceram um parcelamento da fatura do cartão...


### <font color='green'> Remoção de pontuações e deixando o texto em caixa baixa

In [8]:
def processa_sentimento(text):
 # Minuscula
 text = text.lower()

 # retirando pontuações, exceto "."
 text = re.sub(r'[^\w\s.]', '', text)

 return text

In [9]:
dados_financeiros['Relato'] = dados_financeiros['Relato'].apply(processa_sentimento)

C:\Users\alyss\AppData\Local\Temp\ipykernel_20312\750078115.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_financeiros['Relato'] = dados_financeiros['Relato'].apply(processa_sentimento)


In [11]:
dados_financeiros.sample(5)

,Empresa,Relato
5217,C6 Bank,venho solicitar o ded demonstrativo de evoluç...
2544,Nubank,minha dívida era de 35000hoje está em 334100 p...
8104,Mercado Pago,no dia 06 de outubro do corrente ano de 2024 a...
2420,Caixa Econômica Federal,consta registro de negativação referente contr...
7423,Banco Santander,minha divida era r 1.50000 e hoje está em r 2....


In [30]:
dados_financeiros.to_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/dados_financeiros_sentimentos.csv',
                       sep =',', index = False, encoding = 'utf-8')


### <font color='green'> Pré processamento para frequência